### Import Statements

In [ ]:
import keras.preprocessing.text as t
from keras.preprocessing.sequence import pad_sequences
import numpy as np
import os
import pickle
from keras.layers import Input, LSTM, Embedding, Dense
from keras.models import Model
from keras.utils import plot_model

/Users/shweta/anaconda/lib/python2.7/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


### Clean the data [movie_lines] to extract just the lines and also create a dictionary with dialogue id as key and dialogue as value

In [2]:
dialogue_conversation_exists = os.path.exists(os.path.join('data', 'dialogue_conversation'))
movie_lines_exists = os.path.exists(os.path.join('data', 'movie_lines.txt'))

if not (dialogue_conversation_exists & movie_lines_exists):
    raw_movie_lines = open(os.path.join('data', 'movie_lines.txt'), 'r').read().split('\n')[:-1]
    dialogue_conversation = {}
    
    with open(os.path.join('data','just_movie_lines.txt'), 'w') as f:
        for line in raw_movie_lines:
            line = line.split(' +++$+++ ')
            dialogue_id = line[0]
            conversation = line[-1]
            f.write(conversation + '\n')
            dialogue_conversation[dialogue_id] = conversation
    
    pickle.dump(dialogue_conversation, open(os.path.join('data', 'dialogue_conversation'), 'wb'), True)
else:
    dialogue_conversation = pickle.load(open(os.path.join('data', 'dialogue_conversation'), 'rb'))


### Extract the Embedding Indices from Pre-trained model

In [3]:
embeddings_index = {}
if not os.path.exists(os.path.join('data', 'embeddings_index')):
    f = open(os.path.join('glove.6B', 'glove.6B.100d.txt'))
    for line in f:
        values = line.split()
        word = values[0]
        coeffs = np.asarray(values[1:], dtype='float32')
        embeddings_index[word] = coeffs
    f.close()
    
    pickle.dump(embeddings_index, open(os.path.join('data', 'embeddings_index'), 'wb'), True)
else:
    embeddings_index = pickle.load(open(os.path.join('data', 'embeddings_index'), 'rb'))


### Tokenize the dataset to extract words

In [4]:
lines = open(os.path.join('data','just_movie_lines.txt'), 'r').read().split('\n')[:-1]
min_count = 15
tokenizer = t.Tokenizer(lines)
tokenizer.fit_on_texts(lines) 
num_words = sum([1 for _, v in tokenizer.word_counts.items() if v >= min_count])

In [5]:
print('Current vocabulary after choosing only most frequent words', str(num_words))

('Current vocabulary after choosing only most frequent words', '8424')


In [6]:
tokenizer = t.Tokenizer(num_words=num_words)
# Assigns id to words in the lines according to word count
tokenizer.fit_on_texts(lines) 

# word_index is a dictionary of word and its index.
word_index = tokenizer.word_index

### Word to Index and Index to Word Dictionary

In [9]:
word_to_index = {key: word_index[key] + 3 for key in word_index if word_index[key] <= num_words}
index_to_word = {word_to_index[key]: key for key in word_to_index}

word_to_index['<pad>'] = 0
word_to_index['<bos>'] = 1
word_to_index['<eos>'] = 2
word_to_index['<unk>'] = 3

index_to_word[0] = '<pad>'
index_to_word[1] = '<bos>'
index_to_word[2] = '<eos>'
index_to_word[3] = '<unk>'

### Extracting Conversations

In [10]:
conversations = []
conversations_exists = os.path.exists(os.path.join('data', 'conversations'))

if not conversations_exists:
    raw_movie_conversations = open(os.path.join('data', 'movie_conversations.txt'), 'r').read().split('\n')[:-1]
    
    # Extracting the conversation list and forming a list of conversations 
    # Here con_a is previous two lines, con_a_2 is current line and con_b is next/target line.
    for conversation in raw_movie_conversations:
        conversation = conversation.split(' +++$+++ ')[-1]
        conversation = conversation.replace('[', '')
        conversation = conversation.replace(']', '')
        conversation = conversation.replace('\'', '')
        conversation = conversation.split(', ')
        
        con_a_1 = ''
        for i in range(len(conversation)-1):
            
            con_a_2 = dialogue_conversation[conversation[i]]
            con_b = dialogue_conversation[conversation[i+1]]
            
            if len(con_a_1.split()) <= 50 and len(con_a_2.split()) <= 50 and len(con_b.split()) <= 50:
                con_a = "{} {}".format(con_a_1, con_a_2)
                conversations.append((con_a, con_b, con_a_2))
            
            con_a_1 = con_a_2
    pickle.dump(conversations, open(os.path.join('data', 'conversations'), 'wb'), True)
else:
    conversations = pickle.load(open(os.path.join('data', 'conversations'), 'rb'))
print(conversations[1])

("Can we make this quick?  Roxanne Korrine and Andrew Barrett are having an incredibly horrendous public break- up on the quad.  Again. Well, I thought we'd start with pronunciation, if that's okay with you.", 'Not the hacking and gagging and spitting part.  Please.', "Well, I thought we'd start with pronunciation, if that's okay with you.")


### Tokenize conversations and add padding ``<pad>``, ``<eos>``, ``<bos>`` and replace out of vocabulary words with ``<unk>``

Maximum number of words in a sentence is 50 

In [11]:
from keras.preprocessing.text import text_to_word_sequence
from keras.preprocessing.sequence import pad_sequences

max_length = 50
vocab = [w_v for w_v in word_index if word_index[w_v]< num_words]

# These are not question and answers but a conversation. 
# Just for convenience sake, I used question and answer as variable names.
question = []
answer = []

question_exists = os.path.exists(os.path.join('data', 'question'))
answer_exists = os.path.exists(os.path.join('data', 'question'))

if not (question_exists & answer_exists):
    for conv in conversations:
        conversation_a = conv[0]
        conversation_b = conv[1]
    
        conversation_a = text_to_word_sequence(conversation_a)
        conversation_b = text_to_word_sequence(conversation_b)
    
        conversation_a.insert(0, '<bos>')
        conversation_a.append('<eos>')
        conversation_b.insert(0, '<bos>')
        conversation_b.append('<eos>')
    
        conversation_a = [word_to_index[c] if c in vocab else 3 for c in conversation_a]
        conversation_b = [word_to_index[c] if c in vocab else 3 for c in conversation_b]
        
        question.append(conversation_a[:max_length])
        answer.append(conversation_b[:max_length])
    
    question = pad_sequences(question, max_length, padding='pre')
    answer = pad_sequences(answer, max_length, padding='post')
    for i in range(answer.shape[1]):
        answer_target
    
    pickle.dump(question, open(os.path.join(os.path.join('data', 'question')), 'wb'), True)
    pickle.dump(answer, open(os.path.join(os.path.join('data', 'answer')), 'wb'), True)
else:
    question = pickle.load(open(os.path.join(os.path.join('data', 'question')), 'rb'))
    answer = pickle.load(open(os.path.join(os.path.join('data', 'answer')), 'rb'))

print(question.shape)
print(answer.shape)


(213602, 50)
(213602, 50)


In [44]:
# +4 for <eos>,<bos>, <unk> and <pad>
# -1 in answer[k][j]-1 because fit_to_text assigns id from 1 to num_words 
# Here, we are setting 1 to the index of next word in the sentence because softmax 
# should predict that word as next word. So, high probability = 1.
answer_target = np.zeros((answer.shape[0], max_length, len(vocab) + 4), dtype='float32')
print answer_target.shape
for k in range(0, answer.shape[0]):
    for j in range(0, answer.shape[1]):
        if j > 0:
            answer_target[k][j-1][answer[k][j]-1] = 1

(213602, 50, 8423)
(213602, 50, 8423)
(213602, 50, 8423)
(213602, 50, 8423)
(213602, 50, 8423)
(213602, 50, 8423)
(213602, 50, 8423)
(213602, 50, 8423)
(213602, 50, 8423)
(213602, 50, 8423)
(213602, 50, 8423)
(213602, 50, 8423)
(213602, 50, 8423)
(213602, 50, 8423)
(213602, 50, 8423)
(213602, 50, 8423)
(213602, 50, 8423)
(213602, 50, 8423)
(213602, 50, 8423)
(213602, 50, 8423)
(213602, 50, 8423)
(213602, 50, 8423)
(213602, 50, 8423)
(213602, 50, 8423)


KeyboardInterrupt: 

### Preparing Embedding Matrix

In [69]:
embedding_matrix = np.zeros((len(vocab) + 1, 100))answer_target=np.zeros((answer.shape[0],max_length,len(vocab)+4),dtype='float32')
print answer_target.shape
for k in range(0, answer.shape[0]):
    for j in range(0, answer.shape[1]):
        if j > 0:
            answer_target[k][j-1][answer[k][j]-1] = 1
for word, i in word_index.items():
    if word in vocab:
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None:
            # words not found in embedding index will be all-zeros.
            embedding_matrix[i] = embedding_vector
print("Embedding Matrix shape",embedding_matrix.shape)

('Embedding Matrix shape', (8424, 100))


In [70]:
embedding_size=100
encoder_inputs = Input(shape=(max_length,))
#Embedding(Size_Of_Vocab, Size_Of_Embedding_Vector, weights=[embedding_matrix] Input_Length)
#Vocab_Size(rows)* embedding_vector_size(columns) must be equal to the embedding_matrix size) 
#enc_embedding_layer = Embedding(num_words+4, embedding_size,input_length=max_length)(encoder_inputs)
#uncomment this line to include the embedding matrix

enc_embedding_layer=  Embedding(num_words,embedding_size,weights=[embedding_matrix],input_length=max_length)(encoder_inputs)
encoder = LSTM(100, return_state=True)
_, state_h, state_c = encoder(enc_embedding_layer)
# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]

In [71]:
# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = Input(shape=(max_length,))
dex=Embedding(num_words,embedding_size,input_length=max_length)

final_dex= dex(decoder_inputs)
#uncomment this line to include the embedding_matrix
#dec_embedding_layer = Embedding(55843, embedding_size,weights=[embedding_matrix],input_length=max_length)(decoder_inputs)
decoder_lstm = LSTM(100, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(final_dex,initial_state=encoder_states)
decoder_dense = Dense(num_words, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['acc'])
model.summary()


__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_20 (InputLayer)           (None, 50)           0                                            
__________________________________________________________________________________________________
input_21 (InputLayer)           (None, 50)           0                                            
__________________________________________________________________________________________________
embedding_4 (Embedding)         (None, 50, 100)      842400      input_20[0][0]                   
__________________________________________________________________________________________________
embedding_5 (Embedding)         (None, 50, 100)      842400      input_21[0][0]                   
__________________________________________________________________________________________________
lstm_4 (LS

In [72]:
model.compile(optimizer='rmsprop', loss='categorical_crossentropy',metrics=['acc'])
model.fit([question, answer], answer_target,
          batch_size=128,
          epochs=1,
          validation_split=0.2)

Train on 170881 samples, validate on 42721 samples
Epoch 1/1


KeyboardInterrupt: 

'''Uncomment this if the training to the model is required
encoder_input_data= conversation-dialogue 1 dataset
decoder_input_data= conversation-dialogue 2 dataset
decoder_target_data= conversation-dialogue 2 dataset

Example: 
Creating the array of zeros with the required shape and populate these arrays with the data.
encoder_input_data = np.zeros(
    (len(lines.eng), 7),
    dtype='float32')
decoder_input_data = np.zeros(
    (len(lines.fr), 16),
    dtype='float32')
decoder_target_data = np.zeros(
    (len(lines.fr), 16, num_decoder_tokens),
    dtype='float32')
   
decoder_target_data is a 3 dimensional array, we need to construct a 3-dimensional array for the dense(softmax layer)

for i, (input_text, target_text) in enumerate(zip(lines.eng, lines.fr)):
    for t, word in enumerate(input_text.split()):
        encoder_input_data[i, t] = input_token_index[word]
    for t, word in enumerate(target_text.split()):
        #decoder_target_data is ahead of decoder_input_data by one timestep
        if t != 16:
            decoder_input_data[i, t] = target_token_index[word]
        if t > 0:
            #decoder_target_data will be ahead by one timestep
            #and will not include the start character.
            decoder_target_data[i, t - 1, target_token_index[word]] = 1.

model.fit([question, answer], decoder_target_data,batch_size=256,epochs=10000,validation_split=0.05)
'''

### Inference
#In order to test or infer the model, encoder and decoder models are required.
#As the encoder model gives the #encoder_states for a particular input which acts as an input to the decoder model

In [73]:
encoder_model = Model(encoder_inputs, encoder_states)
encoder_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_20 (InputLayer)        (None, 50)                0         
_________________________________________________________________
embedding_4 (Embedding)      (None, 50, 100)           842400    
_________________________________________________________________
lstm_4 (LSTM)                [(None, 100), (None, 100) 80400     
Total params: 922,800
Trainable params: 922,800
Non-trainable params: 0
_________________________________________________________________


In [91]:
decoder_state_input_h = Input(shape=(embedding_size,))
decoder_state_input_c = Input(shape=(embedding_size,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

final_dec_emb_layer= dex(decoder_inputs)

decoder_outputs2, state_h2, state_c2 = decoder_lstm(final_dec_emb_layer, initial_state=decoder_states_inputs)
decoder_states2 = [state_h2, state_c2]
decoder_outputs2 = decoder_dense(decoder_outputs2)
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs2] + decoder_states2)

# Reverse-lookup token index to decode sequences back to
# something readable.
#reverse_input_char_index = dict(
#    (i, char) for char, i in input_token_index.items())
# reverse_target_char_index = dict(
#     (i, char) for char, i in target_token_index.items())

In [92]:
def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)
    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1,max_length))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0] = word_to_index['<bos>']

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict(
            [target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_word = index_to_word[sampled_token_index]
        decoded_sentence += ' '+sampled_word

        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_word == '<eos>' or
           len(decoded_sentence) > max_length+2):   #added two for <bos> and <eos>
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1,max_length))
        target_seq[0, 0] = sampled_token_index

        # Update states
        states_value = [h, c]

    return decoded_sentence

In [93]:
for seq_index in [14077,20122,40035,40064, 40056, 40068, 40090, 40095, 40100, 40119, 40131, 40136, 40150, 40153]:
    input_seq = question[seq_index: seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    print('-')
    print('Input sentence:', question[seq_index: seq_index + 1])
    print('Decoded sentence:', decoded_sentence)

-
('Input sentence:', array([[   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    3,  642,   49,
         142,    5,   54,   31,  596,   15,    7,   24, 1762,   28, 1607,
          19,    4,    7,  173,   15,    3]], dtype=int32))
('Decoded sentence:', ' eric eric eric eric eric eric eric eric eric eric eric')
-
('Input sentence:', array([[   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    3,    4,   36,    8,
           3,  366,    3,  366, 1991,    3]], dtype=int32))
('Decoded sentence:', ' eric eric eric eric eric eric eric eric eric eric eric')
-
('Input sentence:', array([[   0,    0,    0,    0,    0,    0,    0,    0,    0